In [21]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [22]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

letter_list = list(alphabet)
letter_list

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [24]:
url_template = 'https://www.pro-football-reference.com/players/{}/'


def get_WR(letter):
    url = url_template.format(letter) # This formats the url by letter

    response = requests.get(url)
    time.sleep(8)

    soup = BeautifulSoup(response.text, 'html.parser')

    players = []
    div_players = soup.find('div', class_="section_content", id="div_players")
    
    # Loop through p tags
    for p in div_players.find_all('p'):
        meta = p.get_text()
        position = str(meta.split('(')[1].split(')')[0])
        is_wr = 1 if 'WR' in position else 0
        year_span = meta.split(')')[1].strip()
        hyper_link = p.find('a')['href']
        player_link = f"'https://www.pro-football-reference.com{hyper_link}"
        if is_wr == 1:
            players.append([meta, position, year_span, player_link])
    df = pd.DataFrame(players, columns = ['meta', 'position', 'year_span', 'player_link'])
    return df 

all_players = pd.concat([get_WR(letter) for letter in letter_list], ignore_index=True)
all_players.to_csv("wide_receiver_links.csv", index = False)
